In [13]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [14]:
import tempfile
import zipfile

temp_dir = tempfile.TemporaryDirectory()
print(temp_dir)

<TemporaryDirectory 'C:\\Users\\andre\\AppData\\Local\\Temp\\tmp1y26oiti'>


In [15]:
with zipfile.ZipFile('dataset.zip', 'r') as zip:
    zip.extractall(temp_dir.name)

In [16]:
#estrutura da rede neural
clf = Sequential()
clf.add(InputLayer(shape=(64, 64, 3)))#imagens 64x64 e coloridas(3)
clf.add(Conv2D(filters= 32, kernel_size=(3,3), activation='relu'))#camadas de convolução(32), kernel_size(tamanho do detector de caracteristicas)
clf.add(BatchNormalization())
clf.add(MaxPooling2D(pool_size=(2,2)))

clf.add(Conv2D(filters= 32, kernel_size=(3,3), activation='relu'))#camadas de convolução(32), kernel_size(tamanho do detector de caracteristicas)
clf.add(BatchNormalization())
clf.add(MaxPooling2D(pool_size=(2,2)))

clf.add(Flatten())

clf.add(Dense(units=128, activation='relu'))
clf.add(Dropout(0.2))
clf.add(Dense(units=128, activation='relu'))
clf.add(Dropout(0.2))

clf.add(Dense(units=1, activation='sigmoid'))#1 pois é classificação binaria

In [17]:
clf.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 62, 62, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 29, 29, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 29, 29, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │       802,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 829,985 (3.17 MB)

 Trainable params: 829,857 (3.17 MB)

 Non-trainable params: 128 (512.00 B)

In [18]:
clf.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [19]:
gerador_treinamento = ImageDataGenerator(rescale=1./255, rotation_range=7,
                                         horizontal_flip=True, shear_range=0.2,
                                         height_shift_range=0.07, zoom_range=0.2)

In [20]:
gerador_teste = ImageDataGenerator(rescale=1./255)

In [21]:
base_treinamento = gerador_treinamento.flow_from_directory(f'{temp_dir.name}/dataset/training_set',
                                                           target_size=(64,64),
                                                           batch_size=32, class_mode='binary')

Found 4000 images belonging to 2 classes.


In [22]:
base_teste = gerador_teste.flow_from_directory(f'{temp_dir.name}/dataset/test_set',
                                                target_size=(64,64),
                                                batch_size=32, class_mode='binary')

Found 1000 images belonging to 2 classes.


In [23]:
clf.fit(base_treinamento, epochs=10, validation_data=base_teste)

Epoch 1/10


c:\Users\andre\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


125/125 ━━━━━━━━━━━━━━━━━━━━ 45s 330ms/step - accuracy: 0.5322 - loss: 0.9993 - val_accuracy: 0.5040 - val_loss: 0.8345
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 18s 139ms/step - accuracy: 0.6164 - loss: 0.6713 - val_accuracy: 0.5150 - val_loss: 1.2206
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 18s 139ms/step - accuracy: 0.6319 - loss: 0.6447 - val_accuracy: 0.6390 - val_loss: 0.6463
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 18s 139ms/step - accuracy: 0.6599 - loss: 0.6163 - val_accuracy: 0.6190 - val_loss: 0.6895
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 18s 139ms/step - accuracy: 0.6786 - loss: 0.5941 - val_accuracy: 0.6490 - val_loss: 0.6049
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 18s 139ms/step - accuracy: 0.6976 - loss: 0.5734 - val_accuracy: 0.6090 - val_loss: 0.6655
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 18s 139ms/step - accuracy: 0.7146 - loss: 0.5552 - val_accuracy: 0.6790 - val_loss: 0.5932
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 18s 139ms/step - accuracy: 0.7275 - loss: 0.5249 - val